# *Statistical Downscaling*

## Pendahuluan

Teknik *downscaling* perlu memperhatikan berbagai variabel iklim pada skala lokal, seperti topografi, karakteristik permukaan tanah, dan lain sebagainya. Terdapat dua cara untuk memperoleh informasi iklim lokal dari data iklim global, yaitu statistik (*Statistical Downscaling*) dan dinamik (*Dynamic Downscaling*). Wilby dan Wigley [@wilby1997] mengelompokkan teknik *downscaling* menjadi 4 kategori: regresi, pola cuaca, stokastik, dan *Limited-area climate models* (LCM). Metode *downscaling* digunakan pada aplikasi dalam bidang hidrologi [@piani2010suhu], pertanian [@glotter2014], dan iklim perkotaan [@smid2018]. Misalnya pada bidang hidrologi sebagai penilaian dampak, teknik *downscaling* digunakan untuk pemodelan debit sungai [@tisseuil2010] dan banjir [@piani2010pr]. Pemilihan teknik *downscaling* salah satunya perlu memperhatikan ketersediaan sumber daya komputasi. Perhitungan pada *statistical downscaling* relatif lebih cepat dibandingkan dengan *dynamical downscaling* sehingga sumber daya komputasi yang dibutuhkan juga relatif lebih murah. Beberapa metode *statistical downscaling* saat ini telah menggunakan berbagai metode *artificial intelligence* seperti *Artificial Neural Network* (ANN) [@hewitson1992large; @buono2010neural; @laddimath2019], *Convolutional Neural Network* (CNN) [@vandal2017deepsd; @bano2020configuration], *Random Forest* [@legasa2022posteriori; @chen2021easy; @pang2017statistical; @yang2017downscaling], dan *Support Vector Machine* [@pour2018model]. Semakin berkembangnya teknologi komputasi saat ini, kedua teknik *downscaling* bahkan dapat dikombinasikan sehingga perolehan informasi iklim lokal diharapkan menjadi lebih akurat (*hybrid downscaling*) [@tran2018; @li2016combined].

## Metode Regresi Linier

Pendekatan *downscaling* dengan metode regresi linier merupakan metode yang paling sederhana dan umum digunakan. Pendekatan ini dapat menggunakan hubungan linier dan juga hubungan non-linier. Contoh hubungan linier misalnya antara suhu udara dengan kelembaban relatif. Hubungan non-linier sering ditemukan pada variabel curah hujan sehingga pemilihan prediktor untuk mengestimasi variabel ini membutuhkan seleksi, misalnya dengan pendekatan *stepwise regression* [@goyal2010evaluation] atau analisis komponen utama (*principal component analysis*, PCA) [@singh2023development; @vu2016statistical]. Secara umum, persamaan regresi linier dapat dituliskan sebagai berikut:

$$
Y = \beta_0 + \beta_1 X + \varepsilon
$$

dengan $Y$ adalah variabel prediktan, $X$ adalah variabel prediktor, $\beta_0$ adalah konstanta, $\beta_1$ adalah koefisien regresi, dan $\varepsilon$ adalah bias/galat. Jika terdapat lebih dari satu variabel prediktor, maka persamaan regresi linier dapat dituliskan sebagai berikut:

$$
Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \ldots + \beta_n X_n + \varepsilon
$$

Perlu diperhatikan bahwa sebelum melakukan regresi linier, prediktor harus memenuhi asumsi, yaitu normalitas, homoskedastisitas, dan independensi. Prediktor yang terdapat pada model iklim global tersedia dalam jumlah banyak sehingga perlu dipilih, salah satu metodenya adalah PCA. Metode PCA umum digunakan untuk menyeleksi prediktor sehingga autokorelasi antar variabelnya dapat diminimalisasi. Selain PCA, metode sederhana, seperti kriteria informasi Akaike (AIC) atau *Variance Inflation Factor* (VIF) juga dapat digunakan untuk menyeleksi prediktor.

## Metode *Artificial Neural Network* (ANN)

Metode *Artificial Neural Network* pertama kali telah diterapkan dengan melakukan *downscaling* curah hujan lokal [@hewitson1992large]. ANN merupakan model yang terdiri dari beberapa lapisan (*layer*) yang terhubung satu sama lain. Lapisan pertama disebut lapisan input, lapisan terakhir disebut lapisan output, dan lapisan di antaranya disebut lapisan tersembunyi (*hidden layer*). Terdapat pembobot (*weight*) yang menghubungkan antar lapisan dan fungsi aktivasi yang menghubungkan antar neuron. Fungsi aktivasi yang umum digunakan adalah fungsi sigmoid, ReLU, dan tanh. Lapisan input dalam hal *downscaling* merupakan beberapa variabel iklim global (suhu, ketinggian geopotensial, kelembaban relatif, kelembaban spesifik, ...) yang digunakan untuk mengestimasi iklim lokal (curah hujan) sebagai output.

![Diagram algoritma ANN. Satu node proses ANN (A) dan Konfigurasi umum proses ANN (B)](Pictures/ann.png)

## Metode *Convolutional Neural Network* (CNN)

Konsep *Convolutional Neural Network* (CNN) mirip dengan ANN. CNN umum digunakan pada data berupa gambar atau video. Penggunaan CNN pada data gambar dapat menjadi keunggulan dalam mengekstraksi fitur spasial yang kompleks sehingga berpotensi digunakan untuk *downscaling* iklim global. CNN memiliki lapisan konvolusi yang digunakan untuk mengekstraksi fitur dari data spasial. Fungsi *loss* di dalam CNN menjadi perhatian penting dalam proses *training data*. Untuk *downscaling* curah hujan harian, fungsi *loss* yang digunakan adalah Gamma-Bernoulli [@rampal2022high].